#개발환경 설정

In [ ]:
!pip3 install -q -U torch==2.3.1
!pip3 install -q -U transformers==4.44.0
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U accelerate==0.33.0

!pip install langchain
!pip install langchain_community
!pip install langchain-teddynote
!pip install sentence-transformers
!pip install faiss-gpu
!pip install pymupdf
!pip install pymupdf4llm
!pip install pdf2docx
!pip install pdfplumber

!pip install FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import torch
from datasets import Dataset, load_dataset, DatasetDict
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline,
                          set_seed,
)
import transformers
from peft import LoraConfig, PeftModel
from accelerate.utils import is_mlu_available

import json
import random
import os
import unicodedata
import pandas as pd
import numpy as np
from tqdm import tqdm
from accelerate import Accelerator
from FlagEmbedding import FlagReranker
import re
import pymupdf
import pymupdf4llm
import pdfplumber
from tabulate import tabulate
from collections import Counter

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader

In [ ]:
import huggingface_hub
huggingface_hub.login("x")
LLAMA_CLOUD_API_KEY= "x"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# MODEL Import

In [ ]:
model = "sh2orc/Llama-3.1-Korean-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/889 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

# Vector DB및 함수 정의

In [ ]:
def extract_table_summarys(load_path):
  """테이블 추출 후 llm을 이용하여 자연어로 정리"""
  with pdfplumber.open(load_path) as pdf:
      texts = ""
      for page in pdf.pages:
          table = page.extract_table()
          print(page)
          if table==None:
            continue
          # RAG 체인 구성
          template = """다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          {}"""
          content = template.format(str(table))
          messages = [{"role": "user", "content": f"{content}"}]
          prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
          terminators = [
                    pipeline.tokenizer.eos_token_id,
                    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
                ]

          # 답변 추론
          print(f"Question: {content}")
          #print(messages)
          #full_response = llm.invoke(prompt)
          outputs = pipeline(prompt,
                            max_new_tokens=500,
                            do_sample=True,
                            temperature=0.2,
                            top_k=3,
                            top_p=0.95,
                            eos_token_id = terminators,
                            )
          summary = outputs[0]["generated_text"][len(prompt):]
          print(summary)
          texts+=summary#여기까지가 파일로드

      splitter = RecursiveCharacterTextSplitter(
          chunk_size=512,
          chunk_overlap=32
      )
      chunks = splitter.split_text(texts)
      return chunks


In [ ]:
def refine_text(text):
  """추출된 텍스트 정보 깔끔하게 정리하기"""
  llama_docs = re.sub(r"-{5,}","\n", text)
  #llama_docs = re.sub(r'\n', r' ',llama_docs)
  llama_docs = re.sub(r"[-*\x07\u20E7\u20E9\u0003\u00B7]", "", llama_docs)
  llama_docs = re.sub(r'[\u2027\u25CB\u3139]', '', llama_docs)
  llama_docs = re.sub(r"\u318d", "", llama_docs)
  llama_docs = re.sub(r"\u25A1", "", llama_docs)
  llama_docs = re.sub(r"\u007C{2,}", "", llama_docs)
  llama_docs = re.sub(r"\u25FE", " ", llama_docs)
  llama_docs = re.sub(r"\u3147", " ", llama_docs)
  llama_docs = re.sub(r"[\u3010]", "[", llama_docs)
  llama_docs = re.sub(r"[\u3011]", "]", llama_docs)
  llama_docs = re.sub(r"\u2018", "'", llama_docs)
  llama_docs = re.sub(r"\u2019", "'", llama_docs)
  llama_docs = re.sub(r"\t", "\n", llama_docs)
  llama_docs = re.sub(r" {3,}", "\n", llama_docs)
  llama_docs = re.sub(r"\n +", "\n", llama_docs)
  llama_docs = re.sub(r" \n ", "\n", llama_docs)
  llama_docs = re.sub(r"\n\n", "\n", llama_docs)
  llama_docs = re.sub(r'\n(.{1})\n', r'\1',llama_docs)
  return llama_docs

In [ ]:
def extract_tables_from_pdf(pdf_path):
  """테이블 추출하기"""
    tables = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            # Extract tables from the page
            page_tables = page.extract_tables()

            for table in page_tables:
                formatted_table = format_table(table)
                tables.append({
                    'page': page_num + 1,
                    'table': formatted_table
                })

    return tables

def format_table(table):
    """테이블 포매팅하기"""
    formatted = []
    for row in table:
        formatted_row = []
        for cell in row:
            if isinstance(cell, list):
                # Nested table case
                formatted_row.append(format_table(cell, refine_text))
            else:
                cell = refine_text(str(cell))
                formatted_row.append(cell)
        formatted.append(formatted_row)
    return formatted

In [ ]:
def remove_tables(load_path, save_path):
  """테이블 pdf에서 지우기"""
    doc = pymupdf.open(load_path)
    for page in doc:
        for tab in page.find_tables():
            # process the content of table 'tab'
            page.add_redact_annot(tab.bbox)  # wrap table in a redaction annotation
            #page.apply_redactions(0, 2, 0)  # erase all table text
    doc.save(save_path)
    # do text searches and text extractions here

In [ ]:
def get_text_refined(load_path):
  """Document 객체에서 텍스트 추출하기"""
  md_docs = pymupdf4llm.to_markdown(load_path)
  text_refined_temp = refine_text(md_docs)

  return text_refined_temp

'\n  doc = pymupdf.open(load_path)\n  text = ""\n  for page in doc:\n    text += page.get_text()\n  refined_text = refine_text(text)\n'

In [ ]:
def get_preprocessed_text(load_path, save_path):
  table_summarys = extract_table_summarys(load_path)
  remove_tables(load_path, save_path)
  text_only = get_text_refined(save_path)
  return text_only, table_summarys

In [ ]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    text_preprocessed , table_summarized = get_preprocessed_text(file_path, "check.pdf")

    # 텍스트를 chunk로 분할

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on = headers_to_split_on, strip_headers = False)
    md_chunks = md_header_splitter.split_text(text_preprocessed)

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_documents(md_chunks)
    #테이블 요약 이어붙이기
    for summary in table_summarized:
        chunks.append(Document(page_content =summary))
    for chunk in chunks:
      print(chunk.page_content)
      print("*"*150)
    return chunks


def create_vector_db(chunks, embeddings):
    """FAISS DB 생성"""

    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory, model_path="intfloat/multilingual-e5-large"):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    # 임베딩 모델 설정
    model_kwargs = {'device': 'cpu'} # 주의
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
            model_name=model_path,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
    )
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks, embeddings)

        #기본 리트리버 생성
        base_retriever = db.as_retriever()

        #sparse리트리버 생성
        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)

        #앙상블 리트리버 생성
        retriever = EnsembleRetriever(
            retrievers = [kiwi_bm25_retriever, base_retriever],
            weights = [0.5, 0.5],
            search_type = "mmr"
        )

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


# DB 생성

In [ ]:
base_directory = '/.data' # Your Base Directory
df = pd.read_csv('/.data/test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...
<Page:1>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['구분', '기금', '소관', '실국(기관)', '계정', '분야', '부문'], ['코드', '중소벤처기업\n창업 및 진흥기금', '중소벤처기업부', '중소기업정책실', '', '110', '118'], ['명칭', None, None, None, None, '산업·중소기업\n및 에너지', '창업 및 벤처']]
다음은 2차원 리스트를 자연어로 표현한 테이블 형태의 정보입니다.

**기금 정보**

- **구분**: 코드, 명칭
- **기금**: 중소벤처기업 창업 및 진흥기금
- **소관**: 중소벤처기업부
- **실국(기관)**: 중소기업정책실
- **계정**: 없음
- **분야**: 산업·중소기업 및 에너지
- **부문**: 창업 및 벤처

이 정보는 모든 항목이 포함되어 있으며, 누락되는 정보가 없습니다.
<Page:2>
<Page:3>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['내역사업명', '구분', '피보조․피출연\n등 기관명', '지원 금액\n(2024 계획안)', '지원 비율(%)', '보조율 법적근거\n(해당 조항)'], ['창업기반지원', '융자', '중소기업', '1조 7,958억원', '정책자금기준금리△0.3%p', '중소기업진흥에\n관한 법률 제66조,\n제66조, 제67조,\n제74조'], ['개발기술사업화', '융자\n+보조', '중소기업', '2,120억원', '(융자)정책자금기준금리\n(이차보전) 최대 5.5% 이내', '중소기업진흥에\n관한 법률 제66조,\n제66조, 제67조,\n제74조']]
다음은 2차원 리스트를 자연어로 표현한 테이블 형태의 정보입

Processing PDFs:  11%|█         | 1/9 [00:35<04:40, 35.04s/it]

Processing 보건복지부_부모급여(영아수당) 지원...
<Page:1>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['구분', '회계', '소관', '실국(기관)', '계정', '분야', '부문'], ['코드', '11\n일반회계', '23\n보건복지부', '인구정책실\n보육정책관', '', '080', '08A'], ['명칭', None, None, None, None, '사회복지', '아동·보육']]
구분은 '코드'와 '명칭'으로 구성되어 있으며, 각 구분은 다음과 같습니다.

- 코드: 11 (일반회계)
- 명칭: None (정보가 누락되어 있습니다. 그러나 '코드'의 '11 (일반회계)'을 참고할 수 있습니다.)

소관은 '23 (보건복지부)'이며, 실국(기관)은 '인구정책실 보육정책관'입니다. 계정은 정보가 누락되어 있습니다. 분야는 '080 (사회복지)'이며, 부문은 '08A (아동·보육)'입니다.
<Page:2>
<Page:3>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['내역사업명', '구분', '피보조․\n피출연 등\n기관명', '지원 금액\n(2024예산)', '지원\n비율(%)', '보조율 법적근거 (해당 조항)'], ['부모급여\n(영아수당)\n지원', '보조', '지자체\n보조', '2,888,694\n백만원', '서울 25~45\n지방 55~75', '보조금 관리에 관한 법률 시행령 제4조\n제1항 (별표 1)']]
부모급여(영아수당) 지원은 보조금으로, 지자체 보조를 통해 지원됩니다. 지원 금액은 2,888,694 백만원이며, 지원 비율은 서울 25~45%, 지방 55~75%입니다. 보조금 관리에 관한 법률 시행령 제4조 제1항 (별표 1)에 근거합니다.
# 사  업  명
(67)

Processing PDFs:  22%|██▏       | 2/9 [00:58<03:18, 28.32s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [11:51<03:50, 115.07s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...
<Page:1>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['', ''], [None, '']]
이 리스트는 2차원 표 형태의 데이터로, 각 행은 두 개의 열을 가지고 있습니다. 첫 번째 행에는 빈 문자열과 빈 문자열이 들어 있으며, 두 번째 행에는 None과''이 들어 있습니다. 

이 데이터를 자연어로 표현하면 다음과 같습니다:

- 첫 번째 행의 첫 번째 열에는 빈 문자열이 들어 있습니다.
- 첫 번째 행의 두 번째 열에는 빈 문자열이 들어 있습니다.
- 두 번째 행의 첫 번째 열에는 값이 누락되어 있습니다.
- 두 번째 행의 두 번째 열에는''이 들어 있습니다.
<Page:2>
<Page:3>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['자율/메타 평가 Assessment', None, None], ['재정사업자율평가', None, '핵심사업평가'], ['기관별 자율 단위사업 점검\n기관별 세출구조조정', None, '국정과제 등 80여개 사업의\n집행관리와 종합평가'], [None, '', ''], [None, '', ''], [None, '', '']]
다음은 자연어로 표현된 테이블 형태의 정보입니다.

1. 자율/메타 평가 Assessment
2. 재정사업자율평가
3. 기관별 자율 단위사업 점검 기관별 세출구조조정
4. 국정과제 등 80여개 사업의 집행관리와 종합평가

이 정보는 총 4개의 항목으로 구성되어 있으며, 각 항목은 재정사업자율평가와 기관별 자율 단위사업 점검 기관별 세출구조조정, 국정과제 등 80여개 사업의 집행관리와 종합평가로 나뉩니다.
<Page:4>
Question: 다음 리스트들은 2차원 표 형태의 데

Processing PDFs:  89%|████████▉ | 8/9 [15:16<02:23, 143.95s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...
<Page:1>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['', ''], [None, '']]
다음은 자연어로 표현된 테이블 형태의 정보입니다.

- 첫 번째 행의 첫 번째 열에는 정보가 없습니다.
- 첫 번째 행의 두 번째 열에는 정보가 없습니다.
- 두 번째 행의 첫 번째 열에는 정보가 없습니다.
- 두 번째 행의 두 번째 열에는'' (공백 문자)가 있습니다.
<Page:2>
<Page:3>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['금액 추정가능성\n자원 유출가능성', '신뢰성있는 추정가능성', '신뢰성있는 추정불가능'], [None, '충당부채로 인식', None], [None, '우발부채로 주석에 공시', None]]
금액 추정가능성 자원 유출가능성은 다음과 같습니다:

1. 신뢰성있는 추정가능성: 충당부채로 인식
2. 신뢰성있는 추정불가능: 우발부채로 주석에 공시

금액 추정가능성 자원 유출가능성은 총 3가지로 나누어집니다. 첫 번째는 신뢰성있는 추정가능성으로, 충당부채로 인식됩니다. 두 번째는 신뢰성있는 추정불가능으로, 우발부채로 주석에 공시됩니다.
<Page:4>
Question: 다음 리스트들은 2차원 표 형태의 데이터입니다. 테이블 형태의 정보를 자연어로 표현해주세요. 누락되는 정보가 없도록 답변하세요.

          [['구분', None, '주요 기준', '설명'], [None, '행정규칙', '「국가회계기준에 관한 규칙」(중앙)\n「지방자치단체 회계기준에 관한\n규칙」(지방)', None], [None, '회계기준', '한국채택국제회계기준(K–IFRS)\n국제공공부문회계기준(IPSAS)', Non

Processing PDFs: 100%|██████████| 9/9 [18:15<00:00, 121.68s/it]


# Langchain 을 이용한 추론

In [ ]:
#GPU 메모리 정리
torch.cuda.empty_cache()

In [ ]:
def find_top_n_indices(numbers, n):
    # 리스트의 인덱스와 값을 함께 저장
    indexed_numbers = enumerate(numbers)

    # 값을 기준으로 내림차순 정렬
    sorted_numbers = sorted(indexed_numbers, key=lambda x: x[1], reverse=True)

    # 상위 n개의 인덱스만 추출
    top_n_indices = [index for index, value in sorted_numbers[:n]]

    return top_n_indices

def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

flagreranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

def format_docs(question, docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""

    reranker = flagreranker
    scores = []
    n = 5
    if n > len(docs):
      n = len(docs)

    for i in range(len(docs)):
      score = reranker.compute_score([question, str(docs[i])], normalize=True)
      scores.append(score)
      indeces = find_top_n_indices(scores, n)

    for index in indeces:
        context += docs[index].page_content
        context += '\n'

    return context



# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """정보의 기준 연도에 유의하여 답변하세요.
숫자 단위에 유의하여 답변하세요.
정확한 정보에 기반하여 답변하세요.
다음 정보를 바탕으로 질문에 답하세요:
{}

### 질문:
{}

질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.

### 답변:
"""



    #Reranker 사용, Chain 체인 과정을 직접 구현
    docs = retriever.invoke(question)
    context = format_docs(question, docs)
    content = template.format(context, question)
    messages = [{"role": "user", "content": f"{content}"}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    terminators = [
              pipeline.tokenizer.eos_token_id,
              pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
          ]

    # 답변 추론
    print(f"Question: {question}")
    #print(messages)
    #full_response = llm.invoke(prompt)
    outputs = pipeline(prompt,
                       max_new_tokens=400,
                       do_sample=True,
                       temperature=0.01,
                       top_k=4,
                       top_p=0.95,
                       eos_token_id = terminators,
                       )
    full_response = outputs[0]["generated_text"][len(prompt):]
    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })
    del full_response

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:02<04:40,  2.89s/it]

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다.

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 2/98 [00:07<06:23,  3.99s/it]

...

Answer: 국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 상이합니다. 국가결산보고서는 우발부채를 재무제표상 부채로 인식하지 않고, 주석에 공시합니다. 반면, 지방자치단체 회계기준은 우발상황, 우발손실, 우발이익으로 정의하고 있으며, 지방정부의 우발부채의 회계처리가 중앙정부와 상이합니다.

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Answering Questions:  99%|█████████▉| 97/98 [10:02<00:07,  7.42s/it]

Answer: 우발부채는 미래의 다양한 의제의무를 포함하는 부채로, 국제기준 재정통계 산출에 발생주의 회계기준이 적용되면서 중요성이 인식됩니다. 우발부채 관리는 지속가능한 재정운영 차원에서 중요한 이슈로, 미래의 다양한 의제의무를 포함하고 있기 때문입니다.

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


Answering Questions: 100%|██████████| 98/98 [10:15<00:00,  6.28s/it]

Answer: 보증(guarantees)은 명시적 우발부채의 가장 보편적 형태이나 모든 보증이 우발부채에 해당하는 것은 아니다. 예컨대 '파생금융상품 형태의 보증'이나 '표준화된 보증 제도하에서 설정하는 충당부채 형태의 보증'은 재정상태표에 부채로 기록한다. 표준화 보증(standardized guarantees)은 통상 아주 적은 금액에 대해 획일적 조건으로 대규모로 발행하는 보증을 말하며, 수출(무역)신용 보증, 환보증, 다양한 종류의 보험(예금, 농작물, 자연재해 등), 농민융자, 모기지론, 학자금융자, 중소기업융자 등이 있다.



# Submission

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv('/.data/sample_submission.csv')

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")   

# 결과를 CSV 파일로 저장
submit_df.to_csv("/.data/llama3.1-ko5.csv", encoding='UTF-8-sig', index=False)

In [ ]:
#메모리 정리
torch.cuda.empty_cache()
import gc
gc.collect()